In [ ]:
compid2name = mapping = {
    "resnet-tensorflow-to-pytorch": "Familiarity W/ Coding Resnet",
    "gift-eval-short-horizon": "GIFT-Eval",
    "ptb-xl-classification-challenge-feature-enhancement": "PTB-XL ECG Classification with feature enhancement",
    "optiver-realized-volatility-prediction-hyperparameter-search": "Optiver Realized Volatility Prediction with Hyperparameter Search",
    "moment-anomaly-detection-score": "MOMENT Anomaly Score calculation",
    "ventilator-pressure-prediction-missing": "Google Brain - Ventilator Pressure Prediction with Missingness",
    "time-series-library-long-horizon-forecast": "Time Series Library Long Horizon Forecast Dataset",
    "child-mind-institute-detect-sleep-states": "Child Mind Institute-- Detect Sleep States",
    "MIT-BIH-Arrhythmia": "MIT-BIH Arrhythmia Detection",
    "hms-harmful-brain-activity-classification": "HMS - Harmful Brain Activity Classification",
    "time-series-library-itransformer-weather-forecast": "Time Series Library iTransformer block implementation",
    "time-series-exam-1-1": "TimeSeriesExam-1",
    "ptb-xl-classification-challenge": "PTB-XL ECG Classification",
    "csdi-implement-model-pm25": "CSDI challenge implement pm2.5",
    "LANL-Earthquake-Prediction": "LANL Earthquake Prediction",
    "ashrae-energy-prediction": "ashrae-energy-prediction",
    "optiver-realized-volatility-prediction-missing": "Optiver Realized Volatility Prediction with Missingness",
    "MIT-BIH-Arrhythmia-Weak-Supervision": "MIT-BIH Arrhythmia Detection with Weak Supervision",
    "online-sales": "Online Product Sales",
    "optiver-realized-volatility-prediction": "Optiver Realized Volatility Prediction",
    "m5-forecasting-accuracy": "M5 Forecasting - Accuracy",
    "amp-parkinsons-disease-progression-prediction": "AMP-Parkinson's Disease Progression Prediction",
    "ventilator-pressure-prediction": "Google Brain - Ventilator Pressure Prediction",
    "sberbank-russian-housing-market": "sberbank-russian-housing-market",
    "stomp-R-to-python": "Familiarity W/ Coding STOMP",
    "ucr-anomaly-detection": "UCR Anolmaly Detection Dataset",
    "recruit-restaurant-visitor-forecasting": "Recruit Restaurant Visitor Forecasting",
    "g2net-gravitational-wave-detection": "G2Net Gravitational Wave Detection",
    "context-is-key-moirai": "context-is-key-moirai",
    "context-is-key-nn5-chronos": "context-is-key-nn5-chronos",
    "ptb-xl-classification-challenge-hyperparameter-search": "PTB-XL ECG Classification with Hyperparameter Search",
    "csdi-reproduce-pm25": "CSDI challenge reproduce pm2.5"
}

compname2id = {v: k for k, v in mapping.items()}

In [23]:
import pandas as pd 
import re 
import os 
from pathlib import Path
run_dir = Path('/raid/xinyul2/runs') 

total_costs = []
comp_cost = dict()
not_found = []
for seed in [0, 1, 2]:
    seed_cost = 0 
    df = pd.read_csv(f'seed{seed}.csv')
    df = df.loc[:31]
    for i, row in df.iterrows():
        compname = row['Challenge'].strip()
        compid = compname2id[compname]
        run_folder_name = row['Run Group Folder Name ']

        #find the run folder name which is compid_{some random encoding}
        comp_folder_name = [f for f in os.listdir(run_dir / run_folder_name) if f.startswith(compid)][0]
        log_path = f'{run_dir}/{run_folder_name}/{comp_folder_name}/run.log'

        pattern = re.compile(r"Total cost after current step: ([0-9.]+) USD")
        last = None

        with open(log_path, "r") as f:
            for line in f:
                m = pattern.search(line)
                if m:
                    last = float(m.group(1))
        
        if compid in comp_cost:
            comp_cost[compid].append(last)
        else:
            comp_cost[compid] = [last]
            
        if last is None:
            not_found.append(f"{compid}_{seed}")
            print(f"Could not find the cost in {log_path}, seed {seed}, compname {compname}")
            continue
        
        seed_cost += last
    total_costs.append(seed_cost)
    print(f"Seed {seed} total cost: {seed_cost:.2f} USD")

Could not find the cost in /raid/xinyul2/runs/2025-05-06T16-04-50-GMT_run-group_aide/online-sales_87b34d50-1314-4a6c-82d4-54ca990c8676/run.log, seed 0, compname Online Product Sales
Could not find the cost in /raid/xinyul2/runs/2025-05-06T16-04-50-GMT_run-group_aide/recruit-restaurant-visitor-forecasting_b109053c-5070-4b5d-8cc2-8975389a45dc/run.log, seed 0, compname Recruit Restaurant Visitor Forecasting
Could not find the cost in /raid/xinyul2/runs/2025-05-06T15-55-42-GMT_run-group_aide/sberbank-russian-housing-market_1c0aec39-bd08-478b-be70-327bd4fb1244/run.log, seed 0, compname sberbank-russian-housing-market
Could not find the cost in /raid/xinyul2/runs/2025-05-06T15-55-42-GMT_run-group_aide/ashrae-energy-prediction_4107d508-072e-4835-bf2d-8752c37b74f1/run.log, seed 0, compname ashrae-energy-prediction
Could not find the cost in /raid/xinyul2/runs/2025-05-06T23-36-54-GMT_run-group_aide/m5-forecasting-accuracy_ccb17e78-0d1d-4a03-8fc2-a2ebff4dab7b/run.log, seed 0, compname M5 Forecas

In [26]:
#fill none with avergae cost 
from copy import deepcopy
comp_cost_ = deepcopy(comp_cost)
for k, v in comp_cost_.items():
    none_count = v.count(None)
    if none_count > 0:
        avg = sum([x for x in v if x is not None]) / (len(v) - none_count)
        for i in range(len(v)):
            if v[i] is None:
                v[i] = avg

seed_cost = [] 
for i in range(3):
    seed_cost.append(sum([comp_cost_[k][i] for k in comp_cost_.keys()]))
    print(f"Seed {i} total cost: {seed_cost[i]:.2f} USD")
print(f"Average cost: {sum(seed_cost) / 3:.2f} USD")

Seed 0 total cost: 66.70 USD
Seed 1 total cost: 59.83 USD
Seed 2 total cost: 59.83 USD
Average cost: 62.12 USD


In [ ]:
#get lite version
lite_names = ["Familiarity W/ Coding Resnet", "MOMENT Anomaly Score calculation",  "Optiver Realized Volatility Prediction with Missingness",
              "MIT-BIH Arrhythmia Detection with Weak Supervision", "PTB-XL ECG Classification with Hyperparameter Search", 
              "Child Mind Institute-- Detect Sleep States"]

lite_cost = []
for i in range(3):
    lite_cost.append(sum([comp_cost_[k][i] for k in comp_cost_.keys() if compid2name[k] in lite_names]))
    print(f"Lite Seed {i} total cost: {lite_cost[i]:.2f} USD")
print(f"Lite Average cost: {sum(lite_cost) / 3:.2f} USD") 

Lite Seed 0 total cost: 7.93 USD
Lite Seed 1 total cost: 7.98 USD
Lite Seed 2 total cost: 7.98 USD
Lite Average cost: 7.96 USD
